In [2]:
from dotenv import load_dotenv

In [3]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

In [4]:
base_url = "http://localhost:11434"
model = "llama3.2"

llm = ChatOllama(base_url=base_url,
                 model=model,
                 temperature=0.5)

## Sentiment Chain

In [5]:
sentiment_template = """Sentiment Classification Instructions:

Review Analysis Criteria:
- Determine overall sentiment of the review
- Use ONLY the provided text
- Classify strictly as 'Positive' or 'Negative'
- Base decision on:
  * Emotional tone
  * Explicit sentiment indicators
  * Overall customer experience

Review: {review}

Output Requirements:
- Single word response
- Exact match: 'Positive' or 'Negative'
- No additional explanation
- No punctuation

Classification:"""

sentiment_prompt = ChatPromptTemplate.from_template(template=sentiment_template)

sentiment_chain = sentiment_prompt | llm | StrOutputParser()

In [6]:
type(sentiment_chain)

langchain_core.runnables.base.RunnableSequence

In [11]:
sentiment_chain.invoke({"review": "Thank you so much for providing such a great plateform for learning. I am really happy with the service"})

'Positive'

## Postive Chain

In [7]:
positive_template = """Customer Appreciation Response Guidelines:

Response Objectives:
- Express genuine gratitude
- Create emotional connection
- Encourage social media sharing
- Reinforce positive brand experience

Response Components:
1. Personalized acknowledgment
2. Specific reference to review highlights
3. Invitation to share experience
4. Social media sharing encouragement

Tone Requirements:
- Warm and authentic
- Enthusiastic
- Professional
- Conversational

Key Sharing Platforms:
- Instagram
- Twitter/X
- LinkedIn
- Facebook

Review: {review}
"""

positive_prompt = ChatPromptTemplate.from_template(template=positive_template)

positive_chain = (
    positive_prompt | llm | StrOutputParser
)

In [9]:
type(positive_chain)

langchain_core.runnables.base.RunnableSequence

## Negative Chain

In [10]:
negative_template = """Negative Review Response Protocol:

Response Objectives:
- Demonstrate genuine empathy
- Take full accountability
- Provide clear resolution pathway
- Rebuild customer trust

Response Framework:
1. Sincere, personal apology
2. Acknowledge specific concerns
3. Offer concrete resolution steps
4. Provide direct communication channel

Tone Requirements:
- Empathetic
- Professional
- Solution-oriented
- Non-defensive

Communication Guidelines:
- Validate customer's experience
- Avoid blame or justification
- Show commitment to improvement
- Offer private follow-up

Customer Engagement:
- Direct email for detailed discussion
- Propose specific resolution
- Request further details
- Optional compensation/remedy

Contact Information:
Customer Care Email: abc.info@gmail.com

Review: {review}
"""

negative_prompt = ChatPromptTemplate.from_template(template=negative_template)

negative_chain = negative_prompt | llm | StrOutputParser()

In [11]:
def route(info):
    if "positve" in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [12]:
feedback_chain = (
    {'sentiment': sentiment_chain, 'review': lambda x:x['review']}
    |RunnableLambda(route)
)

In [13]:
type(feedback_chain)

langchain_core.runnables.base.RunnableSequence

In [36]:
feedback_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template="Sentiment Classification Instructions:\n\nReview Analysis Criteria:\n- Determine overall sentiment of the review\n- Use ONLY the provided text\n- Classify strictly as 'Positive' or 'Negative'\n- Base decision on:\n  * Emotional tone\n  * Explicit sentiment indicators\n  * Overall customer experience\n\nReview: {review}\n\nOutput Requirements:\n- Single word response\n- Exact match: 'Positive' or 'Negative'\n- No additional explanation\n- No punctuation\n\nClassification:"), additional_kwargs={})])
             | ChatOllama(model='llama3.2', temperature=0.5, base_url='http://localhost:11434')
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(route)

In [37]:
review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."

feedback_reply = feedback_chain.invoke(
    input={'review': review}
)

print(feedback_reply)

Dear [Reviewer's Name],

I wanted to personally reach out and express my sincerest gratitude for taking the time to share your wonderful experience with our platform. It truly warms my heart to hear that you're happy with the service we provide, and I'm thrilled to have played a part in helping you learn and grow.

Your kind words mean the world to us, and I want to assure you that we take pride in offering a high-quality learning environment that supports our customers' success. Your satisfaction is our top priority, and I'm grateful for your feedback.

However, I did want to briefly discuss some minor concerns you may have had during your experience. If there's anything specific that didn't quite meet your expectations or if you encountered any difficulties, please don't hesitate to share them with me directly at abc.info@gmail.com. Your input is invaluable in helping us improve and refine our services.

As a token of appreciation for your loyalty and trust, I'd like to propose the f

In [38]:
review = "I am not happy with the service. It is not good."

feedback_reply = feedback_chain.invoke(
    input={'review': review}
)

print(feedback_reply)

Dear valued customer,

I am truly sorry to hear that you are not satisfied with our service. I can imagine how frustrating it must be to experience a subpar experience, and for that, I apologize personally. My name is [Your Name], and I will do everything in my power to make things right.

You expressed concerns about the quality of our service, and I want to acknowledge those specific issues. Please know that your feedback is invaluable to us, and we take all complaints seriously. We are committed to continuously improving our services to ensure that every customer has an exceptional experience.

To address your concerns, I would like to propose the following concrete steps:

1. [Specific action or resolution you plan to take]
2. [Specific action or resolution you plan to take]

I want to assure you that we value your trust and are committed to rebuilding it. We will do everything possible to prevent similar issues in the future.

If you would like to discuss this further, I invite yo

***

In [1]:
from typing import Dict, Any
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda,RunnablePassthrough


class FeedbackAnalyzer:
    """
    FeedbackAnalyzer processes customer reviews to classify sentiment
    and generate an appropriate response using LangChain's LLM tools.
    
    Attributes:
        base_url (str): The base URL for the LLM model.
        model_name (str): The name of the LLM model to use.
        temperature (float): Temperature setting for the LLM model.
    """
    def __init__(self, base_url: str, model_name: str, temperature: float = 0.5):
        self.base_url = base_url
        self.model_name = model_name
        self.temperature = temperature
        self.llm = ChatOllama(
            base_url=self.base_url,
            model=self.model_name,
            temperature=self.temperature
        )
        self.sentiment_chain = self._build_sentiment_chain()
        self.positive_chain = self._build_positive_chain()
        self.negative_chain = self._build_negative_chain()

    def _build_sentiment_chain(self) -> RunnableLambda:
        """
        Creates the sentiment classification chain.

        Returns:
            RunnableLambda: The sentiment classification chain.
        """
        template = """Sentiment Classification Instructions:

        Review Analysis Criteria:
        - Determine overall sentiment of the review
        - Use ONLY the provided text
        - Classify strictly as 'Positive' or 'Negative'
        - Base decision on:
          * Emotional tone
          * Explicit sentiment indicators
          * Overall customer experience

        Review: {review}

        Output Requirements:
        - Single word response
        - Exact match: 'Positive' or 'Negative'
        - No additional explanation
        - No punctuation

        Classification:"""
        prompt = ChatPromptTemplate.from_template(template=template)
        return prompt | self.llm | StrOutputParser()

    def _build_positive_chain(self) -> RunnableLambda:
        """
        Creates the positive response generation chain.

        Returns:
            RunnableLambda: The positive response chain.
        """
        template = """Customer Appreciation Response Guidelines:

        Response Objectives:
        - Express genuine gratitude
        - Create emotional connection
        - Encourage social media sharing
        - Reinforce positive brand experience

        Response Components:
        1. Personalized acknowledgment
        2. Specific reference to review highlights
        3. Invitation to share experience
        4. Social media sharing encouragement

        Tone Requirements:
        - Warm and authentic
        - Enthusiastic
        - Professional
        - Conversational

        Key Sharing Platforms:
        - Instagram
        - Twitter/X
        - LinkedIn
        - Facebook

        Review: {review}

        Output Requirements:
        - Dont start the reply with `Here's a response that follows the guidelines:`.
        - Just write the final response for user, in a structured format
        """
        prompt = ChatPromptTemplate.from_template(template=template)
        return prompt | self.llm | StrOutputParser()

    def _build_negative_chain(self) -> RunnableLambda:
        """
        Creates the negative response generation chain.

        Returns:
            RunnableLambda: The negative response chain.
        """
        template = """Negative Review Response Protocol:

        Response Objectives:
        - Demonstrate genuine empathy
        - Take full accountability
        - Provide clear resolution pathway
        - Rebuild customer trust

        Response Framework:
        1. Sincere, personal apology
        2. Acknowledge specific concerns
        3. Offer concrete resolution steps
        4. Provide direct communication channel

        Tone Requirements:
        - Empathetic
        - Professional
        - Solution-oriented
        - Non-defensive

        Communication Guidelines:
        - Validate customer's experience
        - Avoid blame or justification
        - Show commitment to improvement
        - Offer private follow-up

        Customer Engagement:
        - Direct email for detailed discussion
        - Propose specific resolution
        - Request further details
        - Optional compensation/remedy

        Contact Information:
        Customer Care Email: abc.info@gmail.com

        Review: {review}

        Output Requirements:
        - Dont start the reply with `Here's a response that follows the guidelines:`.
        - Just write the final response for user, in a structured format
        """
        prompt = ChatPromptTemplate.from_template(template=template)
        return prompt | self.llm | StrOutputParser()

    def _route_response(self, sentiment: str) -> RunnableLambda:
        """
        Routes the feedback chain based on sentiment.

        Args:
            sentiment (str): The sentiment classification ('Positive' or 'Negative').

        Returns:
            RunnableLambda: The appropriate response chain.
        """
        if sentiment.lower() == "positive":
            return self.positive_chain
        else:
            return self.negative_chain

    def process_feedback(self, review: str) -> str:
        """
        Processes a customer review and generates a response.

        Args:
            review (str): The customer review text.

        Returns:
            str: The generated response.
        """
        try:
            feedback_chain = {
                "sentiment": self.sentiment_chain,
                "review": RunnablePassthrough()
            } | RunnableLambda(lambda info: self._route_response(info["sentiment"]))

            return feedback_chain.invoke(input={"review": review})
        except Exception as e:
            raise RuntimeError(f"Error processing feedback: {e}")


if __name__ == "__main__":
    # Example Usage
    try:
        analyzer = FeedbackAnalyzer(base_url="http://localhost:11434", model_name="llama3.2")
        review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
        feedback_reply = analyzer.process_feedback(review)
        print(feedback_reply)
    except RuntimeError as e:
        print(e)


Dear valued customer,

We want to extend our heartfelt gratitude for taking the time to share your wonderful experience with us. It truly means the world to know that you're happy with the service we've provided, and we're thrilled to have had the opportunity to support your learning journey.

Your enthusiasm is contagious, and we're so glad to hear that you've found our platform to be an invaluable resource. We're committed to delivering exceptional experiences like yours, and we appreciate your feedback in helping us achieve this goal.

We'd love for you to share your story with others! Your review on [review platform] has inspired us, and we hope it will do the same for others who are looking for a great learning experience. Would you mind sharing a photo or a brief description of your time with us on social media? We'd be delighted to see how our service has made a positive impact on your life.

Thank you again for being an amazing customer! Your loyalty and support mean everything